# 랭체인 멀티턴

- LangChain의 RunnableWithMessageHistory를 사용하여 대화기록을 관리하는 챗봇 구현
- 세션 ID별로 대화 기록이 어떻게 분리되어 저장되는지 확인

## 라이브러리 불러오기

In [1]:
from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage

c:\ProgramData\anaconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## API KEY 불러오기

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

## 대화 기록 함수 정의

In [4]:
model = ChatOpenAI(model="gpt-4o-mini")

# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(model, get_session_history)

## 첫 번째 대화

In [5]:
config = {"configurable": {"session_id": "abc2"}}  # 세션 ID를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 김진형이야.")],
    config=config,
)

print(response.content)

안녕하세요, 김진형님! 만나서 반갑습니다. 어떻게 도와드릴까요?


## 두 번째 대화

In [6]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 김진형이십니다. 다른 질문이나 부탁이 있으신가요?


## 세션 테스트

In [7]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'죄송하지만, 당신의 이름은 제가 알 수 없습니다. 당신의 이름을 말씀해 주시면 좋겠습니다!'

In [8]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

'우리는 당신의 이름이 김진형이라는 것에 대해 이야기했습니다. 그 외에 다른 주제에 대해 이야기하고 싶으신가요?'

## 스트리밍 응답 출력

In [9]:
config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="")
    # print(r.content, end="|")

김진형이라는 이름은 한국에서 흔히 사용되는 이름이기 때문에, 한국 분이신 것으로 추측해봅니다. 

한국의 문화는 깊고 다양한 전통과 현대가 공존하는 특색이 있습니다. 예를 들어:

1. **음식**: 김치, 비빔밥, 불고기 등 다양한 전통 음식이 있으며, 최근에는 한식이 세계적으로 인기를 끌고 있습니다.

2. **전통 예술**: 한국의 전통 무용, 음악 (예: 국악), 그리고 서예와 같은 예술 형태가 있습니다. 사물놀이와 같은 전통 타악기 공연도 유명합니다.

3. **명절**: 설날과 추석 같은 명절을 통해 가족과 함께 시간을 보내고 조상을 기리는 풍습이 있습니다. 이때 특별한 음식을 준비하고 차례를 지냅니다.

4. **K-pop과 드라마**: 최근 몇 년 사이에 K-pop과 한국 드라마가 전 세계적으로 큰 인기를 끌며 한국 문화를 알리는 데 큰 역할을 하고 있습니다.

5. **한글**: 세계에서 가장 과학적이고 독창적인 글자 중 하나로 평가받는 한글이 있습니다. 세종대왕이 창제했으며, 매년 10월 9일 한글날로 기념합니다.

더 궁금한 점이나 특정한 문화에 대해 알고 싶으신 것이 있으면 말씀해 주세요!